In [1]:
#https://levelup.gitconnected.com/painless-classification-model-using-rnn-b90cb0982543

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import pandas as pd
from nltk import word_tokenize

df = pd.read_csv('/content/drive/MyDrive/MSc DS_NLP/judge-1377884607_tweet_product_company.csv',encoding='ISO-8859-1') 
df = df.rename(columns={"is_there_an_emotion_directed_at_a_brand_or_product": "emotion"})
df = df.dropna(subset=['emotion', 'tweet_text'])

data = df['tweet_text'].map(word_tokenize).values
total_vocabulary = set(word.lower() for tweet in data for word in tweet)  # set created from nested comprehension 
print('There are {} unique words in the dataset.'.format(len(total_vocabulary)))
print('There are {} unique tweets in the dataset.'.format(len(data)))


df.head()

FileNotFoundError: ignored

In [ ]:
def remove_at(text):
  spl_pattern=r'@\w+'
  without_spl=re.sub(pattern=spl_pattern,repl=" ",string=text)
  return without_spl
df['tweet_text']=df['tweet_text'].apply(remove_at)

In [ ]:
df.head()

In [ ]:
import re
def remove_special(text):
  spl_pattern=r'[^a-zA-Z]'
  without_spl=re.sub(pattern=spl_pattern,repl=" ",string=text)
  return without_spl

df['tweet_text']=df['tweet_text'].apply(remove_special)

In [ ]:
df.head()

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

# set the emotion/sentiment as our target
target = df['emotion']

# use one hot encoding since our target is categorical
y = pd.get_dummies(target).values

# use keras to create a Tokenizer object
tokenizer = text.Tokenizer(num_words=20000)  # limit to the num_words most important ones
tokenizer.fit_on_texts(list(df['tweet_text']))
tokenized_texts = tokenizer.texts_to_sequences(df['tweet_text'])
X = sequence.pad_sequences(tokenized_texts, maxlen=100)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# create my NN model
model = Sequential()

embedding_size = 128
model.add(Embedding(len(total_vocabulary), embedding_size))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # use 4 because we have 4 categories

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary() # check the shape

In [ ]:
history=model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test) # get our predictions
y_pred = (y_pred > 0.5) 
acc = accuracy_score(y_test, y_pred) 
print('Overall accuracy of RNN: {:.3f}'.format(acc))

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'], label=' training data')
plt.plot(history.history['val_loss'], label='validation data')
plt.title('Loss for Text Classification')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label=' training data')
plt.plot(history.history['val_accuracy'], label='validation data')
plt.title('Accuracy for Text Classification')
plt.ylabel('Accuracy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
# try with this dataset
#https://www.kaggle.com/uciml/news-aggregator-dataset